In [1]:
# Python code to solve parts #5a - #5f of the GARCH(1,1) problem.
# The code prints labeled sections with comments "#5a", "#5b", ... "#5f".
# It follows the usual GARCH(1,1) formulas and treats volatilities consistently:
# - daily variances are in per-day units (e.g. (0.01732)**2 for 1.732% per day)
# - annualization uses 252 trading days per year
# - when the user says "1% increase for the current volatility per annum", we treat
#   that as an absolute +0.01 in annual volatility (i.e. from e.g. 0.2 -> 0.21)
# - for the continuous-time mean-reversion rate 'a' we use a = -ln(alpha+beta) (standard mapping).
import math
import numpy as np

# Given GARCH(1,1) parameters (daily)
omega = 0.0000013465
alpha = 0.083394
beta = 0.910116
phi = alpha + beta  # persistence

# Provided initial daily volatility (sigma_0) and convert to variance
sigma0_daily = 0.01732   # 1.732% per day
var0_daily = sigma0_daily**2

# trading days per year
DAYS = 252

# Helper prints for nice percentage formatting
def pct(x):
    return f"{100*x:.6f}%"

print("#5a")
# A) long-term variance (VL) and long-term volatility (annualized and daily)
VL_daily = omega / (1 - alpha - beta)  # long-term daily variance
VL_daily, math.sqrt(VL_daily)
VL_annual = VL_daily * DAYS
VL_annual_vol = math.sqrt(VL_annual)
VL_daily_vol = math.sqrt(VL_daily)

print("Long-term daily variance (VL_daily):", VL_daily)
print("Long-term daily volatility (std, daily):", pct(VL_daily_vol))
print("Long-term annual volatility (std, annualized):", pct(VL_annual_vol))
print()

print("#5b")
# B) Expected daily volatility after n days: E[var_{n}] = VL + phi^n (var0 - VL)
def expected_daily_volatility_after_n_days(n, var0=var0_daily):
    E_var = VL_daily + (phi**n) * (var0 - VL_daily)
    return math.sqrt(E_var), E_var

for n in [10, 500]:
    vol_n, var_n = expected_daily_volatility_after_n_days(n)
    print(f"Expected daily volatility after {n} days: {vol_n:.6f} (=> {pct(vol_n)})")
    print(f"  corresponding expected daily variance: {var_n:.10f}")
print()

print("#5c")
# C) Proof (printed argument): E[var_n] -> VL as n->infty because |phi|<1 (phi = alpha+beta)
print("Proof sketch: For GARCH(1,1), E[var_n] = VL + phi^n (var0 - VL).")
print("Since alpha + beta = phi = {:.6f} < 1, we have phi^n -> 0 as n -> infinity.".format(phi))
print("Therefore E[var_n] -> VL, and so sqrt(E[var_n]) -> sqrt(VL). Hence the expected daily volatility")
print("converges to the long-term daily volatility as n -> infinity.")
print()

print("#5d")
# D) Predicted volatility per annum sigma(T) for option maturity T = h / 252
def sigma_T_from_horizon(h, var0=var0_daily):
    if abs(phi - 1.0) < 1e-12:
        sum_phi = h
    else:
        sum_phi = phi * (1 - phi**h) / (1 - phi)
    total_var = h * VL_daily + (var0 - VL_daily) * sum_phi
    ann_var = (DAYS / h) * total_var  # annualized variance
    ann_vol = math.sqrt(ann_var)
    return ann_vol, ann_var, total_var

for h in [10, 500]:
    ann_vol, ann_var, total_var = sigma_T_from_horizon(h)
    print(f"T = {h}/252 years (h={h} days):")
    print("  Predicted annualized volatility sigma(T):", f"{ann_vol:.6f}", "=>", pct(ann_vol))
    print("  Annualized variance:", ann_var)
    print("  Total horizon (h-day) variance (sum of daily expected variances):", total_var)
print()

print("#5e")
# E) Suppose there is an absolute +1% (=0.01) increase in the current volatility per annum.
delta_sigma_hat_annual = 0.01  # absolute 1% point increase in annual volatility
delta_sigma_hat_daily = delta_sigma_hat_annual / math.sqrt(DAYS)

# Current annual volatility implied by sigma0_daily:
sigma0_annual = sigma0_daily * math.sqrt(DAYS)
var0_daily_from_annual = (sigma0_annual / math.sqrt(DAYS))**2
# New sigma0 when annual increases by 0.01:
sigma0_annual_new = sigma0_annual + delta_sigma_hat_annual
sigma0_daily_new = sigma0_annual_new / math.sqrt(DAYS)
var0_daily_new = sigma0_daily_new**2

print("Current annual volatility (from provided daily 1.732%):", pct(sigma0_annual))
print("Increase in annual vol (absolute):", pct(delta_sigma_hat_annual))
print("Equivalent increase in daily volatility (absolute):", f"{100*delta_sigma_hat_daily:.6f}%")
print()

for h in [10, 500]:
    ann_vol_old, _, _ = sigma_T_from_horizon(h, var0=var0_daily)
    ann_vol_new, _, _ = sigma_T_from_horizon(h, var0=var0_daily_new)
    change = ann_vol_new - ann_vol_old
    print(f"For h={h} days (T={h}/252):")
    print("  sigma(T) before increase:", f"{ann_vol_old:.6f}", "=>", pct(ann_vol_old))
    print("  sigma(T) after +1% annual increase:", f"{ann_vol_new:.6f}", "=>", pct(ann_vol_new))
    print("  Actual change in sigma(T):", f"{change:.6f}", "=>", pct(change))
print()

print("#5f")
# F) Use the approximation formula
a = -math.log(phi)
print("Mean-reversion rate a (approx):", a)

sigma_hat_n_annual = sigma0_annual
for h in [10, 500]:
    T = h / DAYS
    ann_vol, _, _ = sigma_T_from_horizon(h, var0=var0_daily)
    if a * T == 0:
        factor = 1.0
    else:
        factor = (1 - math.exp(-a * T)) / (a * T)
    approx_delta = factor * (sigma_hat_n_annual / ann_vol) * delta_sigma_hat_annual
    print(f"For h={h} (T={T:.6f} yrs):")
    print("  factor (1 - e^{-aT})/(aT):", factor)
    print("  sigma(T):", f"{ann_vol:.6f}", "=>", pct(ann_vol))
    print("  Approximate change delta sigma(T):", f"{approx_delta:.6f}", "=>", pct(approx_delta))
print()

# End of script.


#5a
Long-term daily variance (VL_daily): 0.00020747303543913726
Long-term daily volatility (std, daily): 1.440392%
Long-term annual volatility (std, annualized): 22.865521%

#5b
Expected daily volatility after 10 days: 0.017151 (=> 1.715083%)
  corresponding expected daily variance: 0.0002941509
Expected daily volatility after 500 days: 0.014527 (=> 1.452722%)
  corresponding expected daily variance: 0.0002110401

#5c
Proof sketch: For GARCH(1,1), E[var_n] = VL + phi^n (var0 - VL).
Since alpha + beta = phi = 0.993510 < 1, we have phi^n -> 0 as n -> infinity.
Therefore E[var_n] -> VL, and so sqrt(E[var_n]) -> sqrt(VL). Hence the expected daily volatility
converges to the long-term daily volatility as n -> infinity.

#5d
T = 10/252 years (h=10 days):
  Predicted annualized volatility sigma(T): 0.273458 => 27.345827%
  Annualized variance: 0.07477942337763459
  Total horizon (h-day) variance (sum of daily expected variances): 0.0029674374356204202
T = 500/252 years (h=500 days):
  Predict